
<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Three: Classifier training and performance assessment. </h2>	

In [39]:
def train_test_split(df):
    """Split data into training and testing(validation)"""
    n,d = X.shape
    shuffled = np.arange(n)
    np.random.shuffle(shuffled)
    train_n = int(n*0.8)
    test_n = n-train_n
    train_idx = shuffled[:train_n]
    test_idx = shuffled[train_n:]
    return X.iloc[train_idx], y.iloc[train_idx], X.iloc[test_idx,], y.iloc[test_idx]
    
#Split the data into a training set, and test set 
def accuracy(pred, actual):
    return np.sum(pred==actual)/len(pred)
# Calculate the accuracy percentage of the predicted values

<h3>  Train models using all of the following methods below. Be sure to drop the actual image column, and the encoding</h3>	Take note of the differences in accuracy, and methods.


Logistic Regression

K-nearest Neighbors

Classification Tree

Random Forest

Support Vector Machine